In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
import warnings
import random

# Ignore all warnings
warnings.filterwarnings('ignore')

C:\Users\Maram\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Maram\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
file_path = 'C:/Users/Maram/Desktop/stage/Personalized Training Program generator/json_data/AI_Application.Gym_Exercice.json'
with open(file_path, 'r', encoding='utf-8') as file:
    exercises_df = json.load(file)
exercise_data = pd.DataFrame(exercises_df)
exercise_data

,_id,name,Equipement,muscles_involved,image_src_note,Category,Steps,Tips,Level,Type de Exercice,Mistake to Avoid,Video,Variants,Benefits
0,1,Squat Goblet with Kettlebell,[kettlebell],[quadriceps],{'src': 'https://www.docteur-fitness.com/wp-co...,"[Quadriceps exercises, Gluteal exercises, Exer...",{'How to make the Goblet squat with Kettlebell...,{'Training advice': ['If you have trouble maki...,Intermediate,NaN,NaN,NaN,NaN,NaN
1,2,Australian tractions with suspension straps,[rod],"[back, shoulders, glutes, abdominal belt muscl...",{'src': 'https://www.docteur-fitness.com/wp-co...,[Back exercises],{'How to make Australian pulls with suspension...,{'Training advice': ['To protect your shoulder...,Intermediate,NaN,NaN,NaN,NaN,NaN
2,3,Suspended knee survey,NaN,[abdominals],{'src': 'https://www.docteur-fitness.com/wp-co...,[Abdominal exercises],{'How to make the suspended knee survey?': ['1...,{'Advice for the suspended knee survey': ['The...,Beginner,NaN,NaN,NaN,NaN,NaN
3,4,Squat on bench,[bench],"[back, glutes, quadriceps, hamstrings, abdomin...",{'src': 'https://www.docteur-fitness.com/wp-co...,[Quadriceps exercises],{'How to do the squat on a bench?': ['1: Insta...,{'Training advice': ['Do not switch too much b...,Intermediate,NaN,NaN,NaN,NaN,NaN
4,5,Lifted earth with deficit,NaN,"[back, glutes, hamstrings]",{'src': 'https://www.docteur-fitness.com/wp-co...,[Exercices hamstrings],{'How to make the lifting of earth with defici...,{'Training advice': ['The first thing you need...,Beginner,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552,3553,Goblet Squats,Kettlebells,Quadriceps,"{'title': 'Goblet Squats Images', 'src': ['htt...",NaN,{'Images': 'https://artifacts.bbcomcdn.com/@bb...,NaN,Intermediate,Strength,NaN,NaN,NaN,NaN
3553,3554,Barbell Back Squats,Barbell,Quadriceps,"{'title': 'Barbell Back Squats Images', 'src':...",NaN,{'Images': 'https://artifacts.bbcomcdn.com/@bb...,NaN,Intermediate,Strength,NaN,NaN,NaN,NaN
3554,3555,Reverse Barbell Row,Barbell,Lats,"{'title': 'Reverse Barbell Row Images', 'src':...",NaN,{'Images': 'https://artifacts.bbcomcdn.com/@bb...,NaN,Intermediate,Strength,NaN,NaN,NaN,NaN
3555,3556,Reverse Dumbbell Lunges,NaN,Abdominals,"{'title': 'Reverse Dumbbell Lunges Images', 's...",NaN,{'Images': 'https://artifacts.bbcomcdn.com/@bb...,NaN,Intermediate,Strength,NaN,NaN,NaN,NaN


In [23]:
def process_exercise_data(exercises_df):
    # Convert the list of dictionaries into a DataFrame
    if isinstance(exercises_df, list):  # Assuming the input might be a list of dictionaries
        exercises_df = pd.DataFrame(exercises_df)
    


    # Process 'Equipement'
    exercises_df['Equipement'] = exercises_df['Equipement'].apply(
        lambda x: ['no_need'] if isinstance(x, str) and x.lower() in ['body only','none']
        else [x.lower()] if isinstance(x, str)
        else [i.lower() for i in x] if isinstance(x, list)
        else ['no_need']
    )

    # Process 'muscles_involved'
    exercises_df['muscles_involved'] = exercises_df['muscles_involved'].apply(
        lambda x: [i.lower() for i in x] if isinstance(x, list)
        else [x.lower()] if isinstance(x, str)
        else ['other']  # This will be used for NaNs or any other non-list/non-string entries
    )

    # Process 'Type de Exercice'
    exercises_df['Type de Exercice'] = exercises_df['Type de Exercice'].apply(
        lambda x: [i.lower() for i in x] if isinstance(x, list)
        else [x.lower()] if isinstance(x, str)
        else ['other']
    )

    # Simplify 'Type de Exercice' values
    exercises_df['Type de Exercice'] = exercises_df['Type de Exercice'].apply(
        lambda x: ['cardio' if i == 'cardio exercise' 
                   else 'powerlifting' if i=='powerlifter'
                   else i.lower() for i in x]
    )

    # Process 'Level'
    exercises_df['Level'] = exercises_df['Level'].apply(
       lambda x: 'advanced' if x.lower() == 'expert' 
              else x.lower() if isinstance(x, str) 
              else 'beginner'  # Replace with 'unknown' or another placeholder if the level is not a string
    )
        # Now, add default values for missing columns
    for col in ['Type de Exercice', 'Equipement', 'muscles_involved', 'Level']:
        if col not in exercises_df.columns:
            exercises_df[col] = 'none'
    
    return exercises_df

# Use the function to process the data
processed_exercise_data = process_exercise_data(exercises_df)


In [24]:
# Specify the path to the JSON file where you want to save the data
output_file_path = 'C:/Users/Maram/Desktop/stage/Personalized Training Program generator/json_data/proceed_exercice_file.json'

# Save the DataFrame to a JSON file
processed_exercise_data.to_json(output_file_path, orient='records', lines=True, indent=4)


In [ ]:
from flask import Flask, jsonify, request
import pandas as pd
processed_exercise_data = process_exercise_data(exercises_df)

app = Flask(__name__)

# Ensure you load your processed_exercise_data correctly here
# processed_exercise_data = pd.read_csv('path_to_your_data.csv')

def get_exercises_by_muscle(processed_data, muscle):
    muscle = muscle.lower()  # Convert the muscle parameter to lowercase
    # Filter exercises that involve the specified muscle
    # Checking if the muscle is in any of the list's elements, accounting for case
    result = processed_data[processed_data['muscles_involved'].apply(lambda x: any(muscle in m.lower() for m in x))]
    return result

@app.route('/exercises', methods=['GET'])
def exercises_by_muscle():
    muscle = request.args.get('muscle', None)
    if not muscle:
        return jsonify({'error': 'Muscle parameter is required'}), 400
    try:
        result = get_exercises_by_muscle(processed_exercise_data, muscle)
        if result.empty:
            return jsonify({'message': 'No exercises found for the given muscle'}), 404
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    return jsonify(result.to_dict(orient='records'))

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app, use_reloader=False)


In [26]:
def get_exercises_by_muscle(processed_data, muscle):
    # Ensure the muscle name is in lowercase for matching
    muscle = muscle.lower()
    # Filter exercises that involve the specified muscle
    result = processed_data[processed_data['muscles_involved'].apply(lambda x: muscle in x)]
    return result
exercises_for_quads = get_exercises_by_muscle(processed_exercise_data, 'quadriceps')

# Display the results
print(exercises_for_quads)

       _id                              name                    Equipement  \
0        1      Squat Goblet with Kettlebell                  [kettlebell]   
3        4                    Squat on bench                       [bench]   
19      20            Horizontal thigh press                     [no_need]   
23      24                   Wall Ball Squat                     [no_need]   
31      32  Split Squat at the Smith Machine  [rod, halter, smith machine]   
...    ...                               ...                           ...   
3522  3523              Shuttle run training                     [no_need]   
3527  3528              Banded squat toe tap                       [bands]   
3536  3537               Band overhead squat                       [bands]   
3552  3553                     Goblet Squats                 [kettlebells]   
3553  3554               Barbell Back Squats                     [barbell]   

                                       muscles_involved  \
0   

In [4]:
import pandas as pd
import numpy as np

def process_and_encode_exercice_data(data):
    exercise_data_exploded = data.explode('Equipement').explode('muscles_involved').explode('Type de Exercice')

    exercise_encoded = pd.get_dummies(exercise_data_exploded, columns=['Equipement', 'muscles_involved','Level','Type de Exercice'])

    # Select only the numeric columns from the DataFrame
    exercise_encoded_numeric = exercise_encoded.select_dtypes(include=[np.number, 'bool']).copy()

    exercise_encoded_grouped = exercise_encoded_numeric.groupby('_id').sum().reset_index()

    # Convert values greater than 1 back to 1
    exercise_encoded_binary = exercise_encoded_grouped.copy()

    # Rename the columns as per your requirement
    exercise_encoded_binary.columns = [
        col.replace('muscles_involved_', '')
        if col.startswith('muscles_involved_') else col for col in exercise_encoded_binary.columns
    ]
    exercise_encoded_binary.columns = [
        col.replace('Equipement_', 'Equipment_')
        if col.startswith('Equipement_') else col for col in exercise_encoded_binary.columns
    ]

    exercise_encoded_binary.columns = [
        col.replace('Type de Exercice_', 'Type_')
        if col.startswith('Type de Exercice_') else col for col in exercise_encoded_binary.columns
    ]

    columns_to_clip = exercise_encoded_binary.columns.difference(['_id'])

    # Clip values in the DataFrame, excluding '_id'
    exercise_encoded_binary[columns_to_clip] = exercise_encoded_binary[columns_to_clip].clip(upper=1)
        
    # Save the processed DataFrame to a CSV file
    
    return exercise_encoded_binary

# To use the function:
process_encode_exercice_data=process_and_encode_exercice_data(processed_exercise_data)
process_encode_exercice_data.to_csv('exercice_encoded.csv',index=False)
process_encode_exercice_data.columns

Index(['_id', 'Equipment_bands', 'Equipment_barbell', 'Equipment_bench',
       'Equipment_bike', 'Equipment_cable', 'Equipment_donkey calf machine',
       'Equipment_dumbbell', 'Equipment_e-z curl bar', 'Equipment_elastic',
       'Equipment_exercise ball', 'Equipment_foam roll',
       'Equipment_gymnastics', 'Equipment_halter', 'Equipment_jumping rope',
       'Equipment_kettlebell', 'Equipment_kettlebells', 'Equipment_landmine',
       'Equipment_machine', 'Equipment_medicine ball', 'Equipment_no_need',
       'Equipment_other', 'Equipment_pulley', 'Equipment_rod',
       'Equipment_rowing machine', 'Equipment_smith machine', 'Equipment_step',
       'Equipment_suspension straps', 'Equipment_treadmill',
       'abdominal belt muscles ', 'abdominals', 'abductors', 'adductors',
       'back', 'biceps', 'calves', 'chest', 'deltoides', 'forearm', 'forearms',
       'glutes', 'hamstrings', 'lats', 'lower back', 'lumbar', 'middle back',
       'neck', 'oblique', 'other', 'pectorals', 'q

In [5]:
#users_data=pd.read_csv('user_profiles_dataset.csv')
data = {
    'UserID': [1,2],
    'Age': [29,45],
    'Gender': ['Female','male'],
    'FitnessLevel': ['beginner','Advanced'],
    'Goal': ['','weight loss'],
    'EquipmentAvailable': ['no equipment','Gym'],
    'PreferenceType': ['mixed',''],
    'Days/week':[5,6],
    'TimeAvailable': [40,50],
    'HealthConditions': [['cardiovascular', 'pectorals problems'],[]]
}

users_data=pd.DataFrame(data)

In [6]:
import pandas as pd

def process_user_data(user_df):
    data = pd.DataFrame(user_df) if not isinstance(user_df, pd.DataFrame) else user_df
    
    defaults = {
        'PreferenceType': 'mixed',
        'Goal': 'endurance improvement',
        'FitnessLevel': 'beginner',
        'EquipmentAvailable': 'no_need',
        'HealthConditions': []  # List default
    }
    # Update EquipmentAvailable if set to 'no equipment'
    data['EquipmentAvailable'] = data['EquipmentAvailable'].apply(lambda x: defaults['EquipmentAvailable'] if 'no equipment' in x else x)


    # Ensure all columns are present, filling with defaults if not
    for col, default in defaults.items():
        if col not in data.columns:
            data[col] = pd.Series([default] * len(data))
        else:
            if isinstance(default, list):
                data[col] = data[col].apply(lambda x: default if x is None or (isinstance(x, float) and pd.isna(x)) else x)
            else:
                data[col] = data[col].fillna(default)
                data[col] = data[col].replace('', default)

    # Function to handle the transformation and cleaning of the data entries
    def transform_to_list(x):
        if isinstance(x, str):
            x = x.strip().lower()  # Clean and lower the string
            if ',' in x:
                return [i.strip() for i in x.split(',')]  # Split by commas if present and clean
            else:
                return [x]  # Return the whole string as a single item in a list if no comma
        elif isinstance(x, list):
            # Flatten the list if it contains nested lists and clean strings
            flat_list = []
            for item in x:
                if isinstance(item, list):
                    flat_list.extend(item)  # Extend flat_list with the sublist
                else:
                    flat_list.append(item.strip().lower())
            return flat_list  # Ensure all items are strings and cleaned
        return x  # Return as is if not a string or list

    # Apply transformation to specified columns
    list_columns = ['EquipmentAvailable', 'PreferenceType', 'Goal', 'HealthConditions']
    for col in list_columns:
        data[col] = data[col].apply(transform_to_list)
    
    # Remove duplicates in list columns
    def remove_duplicates(x):
        seen = set()
        seen_add = seen.add
        return [i for i in x if not (i in seen or seen_add(i))]

    for col in list_columns:
        data[col] = data[col].apply(remove_duplicates)

    # Handle 'FitnessLevel' as a single scalar value correctly
    data['FitnessLevel'] = data['FitnessLevel'].apply(lambda x: x.lower() if isinstance(x, str) else defaults['FitnessLevel'])

    return data

# Assuming df_user is the DataFrame you want to process


In [7]:
processed_user_data=process_user_data(data)
processed_user_data

,UserID,Age,Gender,FitnessLevel,Goal,EquipmentAvailable,PreferenceType,Days/week,TimeAvailable,HealthConditions
0,1,29,Female,beginner,[endurance improvement],[no_need],[mixed],5,40,"[cardiovascular, pectorals problems]"
1,2,45,male,advanced,[weight loss],[gym],[mixed],6,50,[]


In [8]:
def process_User_dataframe(df):
   
    
    # Initialize MultiLabelBinarizer and LabelBinarizer
    mlb_equipment = MultiLabelBinarizer()
    mlb_preferences = MultiLabelBinarizer()
    lb_level = MultiLabelBinarizer()
    
    # Fit and transform the data
    equipment_encoded = mlb_equipment.fit_transform(df['EquipmentAvailable'])
    preferences_encoded = mlb_preferences.fit_transform(df['PreferenceType'])
    level_encoded = lb_level.fit_transform(df['FitnessLevel'])
    
    # Convert the encoded results back to a DataFrame
    equipment_df = pd.DataFrame(equipment_encoded, columns=['Equipment_' + e for e in mlb_equipment.classes_])
    preferences_df = pd.DataFrame(preferences_encoded, columns=['Type_' + class_ for class_ in mlb_preferences.classes_])
    level_df = pd.DataFrame(level_encoded, columns=['Level_' + class_ for class_ in lb_level.classes_])
    
    # Concatenate the original DataFrame with the new one-hot encoded DataFrames
    # List of columns to drop
    columns_to_drop = [ 'PreferenceType', 'FitnessLevel', 'Goal', 'Age', 'Gender', 'TimeAvailable', 'HealthConditions', 'EquipmentAvailable']

    # Drop only the columns that exist in the DataFrame
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    final_df = pd.concat([df.drop(columns_to_drop, axis=1), preferences_df, level_df, equipment_df], axis=1)

    # Get all equipment types including 'no_need'
   
    
    # Save the final DataFrame to a CSV file
    #final_df.to_csv(output_csv_path, index=False)
    #final_df.drop('EquipmentAvailable',axis=1,inplace=True)
    # Return the path to the output CSV file
    return final_df

user_encoded=process_User_dataframe(processed_user_data)


In [9]:
user_interaction=pd.read_csv('user_interactions_updated.csv')
user_interaction = user_interaction[user_interaction['user_id'] != 1
                                   ]


In [10]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
#Collaborative Filtering
def generate_item_content_recommendations(user_id, train_set, process_encode_exercice_data, view_weight=0.2, rating_weight=0.8):
    train_set['interaction_score'] = (train_set['view_count'] * view_weight) + (train_set['rating'] * rating_weight)

    # Prepare interaction matrix for the specific user
    user_interactions = train_set[train_set['user_id'] == user_id]
    user_interaction_matrix = pd.pivot_table(user_interactions, index='user_id', columns='exercise_id', values='interaction_score', fill_value=0)

    # Prepare exercise features
    exercise_features = process_encode_exercice_data.set_index('_id')
    common_ids = user_interaction_matrix.columns.intersection(exercise_features.index)
    user_interaction_matrix = user_interaction_matrix[common_ids]
    exercise_features = exercise_features.loc[common_ids]

    # Ensure numeric types
    exercise_features = exercise_features.apply(pd.to_numeric, errors='coerce').fillna(0)
    user_interaction_matrix = user_interaction_matrix.apply(pd.to_numeric, errors='coerce').fillna(0)

    # Compute user profile
    user_profile = user_interaction_matrix.dot(exercise_features)

    # Calculate similarity between user profile and exercise features
    user_item_similarity = cosine_similarity(user_profile, exercise_features)

    # Generate recommendations
    sorted_item_indices = user_item_similarity.argsort()[::-1].flatten()
    
    # Filter out seen items
    seen_items = set(user_interactions['exercise_id'])
    recommendations = [idx for idx in sorted_item_indices if idx not in seen_items]

    return recommendations  # Limit to top N recommendations


In [11]:
health_specific_rules = {
    'cardiovascular': {
        'avoid': ['hiit', 'competition sportsman', 'crossfit'],
        'recommend': ['cardio', 'stretching', 'yoga']
    },
    'respiratory conditions': {
        'avoid': ['hiit', 'crossfit'],
        'recommend': ['yoga', 'stretching']
    },
    'joint issues': {
        'avoid': ['plyometrics', 'hiit', 'olympic weightlifting', 'powerlifting'],
        'recommend': ['stretching', 'yoga', 'body weight']
    },
    'back problems': {
        'avoid': ['plyometrics', 'olympic weightlifting', 'powerlifting', 'crossfit'],
        'recommend': ['stretching', 'yoga', 'calisthenics']
    },
    'diabetes': {
        'recommend': ['cardio', 'strength', 'weight training']
    },
    'obesity': {
        'avoid': ['hiit', 'plyometrics', 'olympic weightlifting'],
        'recommend': ['losing weight', 'cardio', 'body weight']
    },
    'pregnancy': {
        'avoid': ['hiit', 'crossfit', 'olympic weightlifting', 'plyometrics'],
        'recommend': ['yoga', 'stretching']
    },
    'bone conditions': {
        'avoid': ['hiit', 'olympic weightlifting', 'plyometrics'],
        'recommend': ['strength', 'weight training']
    },
    'hypertension': {
        'avoid': ['hiit', 'olympic weightlifting', 'powerlifting'],
        'recommend': ['cardio', 'stretching', 'yoga']
    },
    'neurological conditions': {
        'recommend': ['stretching', 'yoga', 'calisthenics']
    },
    'post-surgery recovery': {
        'avoid': ['hiit', 'crossfit', 'olympic weightlifting', 'plyometrics'],
        'recommend': ['stretching', 'calisthenics', 'yoga']
    },
    # Additional rules can be added as needed
}


In [12]:

#Content-Based Filtering
#both dataset should be processed and encoding
def recommend_based_on_content(user_id, user_encoded_data, exercise_encoded):    # Convert user_profile to DataFrame only if it's not already one
    exercise_ids = exercise_encoded['_id'].copy()
    exercise_features = exercise_encoded.drop(columns=['_id'])
    if not isinstance(user_encoded_data, pd.DataFrame):
        user_encoded_data = pd.DataFrame(user_encoded_data)
    
    # Find the user profile by UserID
    user_profiles = user_encoded_data[user_encoded_data['UserID'] == user_id]

    # If no user profile with the given user_id is found, return an empty list
    if user_profiles.empty:
        raise ValueError(f"No profile found for user_id {user_id}")


    # Ensure user_profile is a single row containing the user's data
    # Make sure that user_profiles only contains the relevant columns
    
    equipment_columns = [col for col in user_profiles.columns if col.startswith('Equipment_')and col != 'Equipment_no_need']
    if 'Equipment_gym' in user_profiles.columns:
            for index, row in user_profiles.iterrows():
                if row.get('Equipment_gym', 0) == 1:
                    for col in equipment_columns:
                        user_profiles.at[index, col] = 1
            user_profiles = user_profiles.drop(columns=['Equipment_gym'])

    type_columns = [col for col in user_profiles.columns if col.startswith('Type_')]
    if 'Type_mixed' in user_profiles.columns:
            for index, row in user_profiles.iterrows():
                if row.get('Type_mixed', 0) == 1:
                    for col in type_columns:
                        user_profiles.at[index, col] = 1
            user_profiles = user_profiles.drop(columns=['Type_mixed'])       
    # Align the columns between user profiles and exercise features
    common_columns = set(exercise_features.columns).intersection(user_profiles.columns)
    missing_in_profiles = set(exercise_features.columns) - common_columns
    missing_in_exercise = set(user_profiles.columns) - common_columns

    for column in missing_in_profiles:
        user_profiles[column] = 0

    for column in missing_in_exercise:
        exercise_features[column] = 0

    # Prepare data by ensuring column order matches
    exercise_features = exercise_features[user_profiles.columns]
    user_profiles.fillna(0, inplace=True)
    exercise_features.fillna(0, inplace=True)

    # Calculate cosine similarity
    similarity_scores = cosine_similarity(user_profiles, exercise_features)

    # Extract top N recommendations for the given user index
    top_indices = np.argsort(-similarity_scores[0])
    recommended_exercise_ids = exercise_ids.iloc[top_indices].tolist()

    return recommended_exercise_ids


## lazem nas2el si on avoid tous les level de ces exercice or que les advanced ones 

In [13]:
import logging

def is_suitable_for_health_condition(health_conditions, exercise_types, involved_muscles):
    suitable = True

    for condition in health_conditions:
        condition_lower = condition.lower()

        if condition_lower in health_specific_rules:
            rules = health_specific_rules[condition_lower]
            # Logging the content of exercise_types to understand its structure
            logging.debug(f"Exercise types: {exercise_types}")
            for exercise_list in exercise_types:
                # Now iterate through each exercise type in the sublist
                for exercise_type in exercise_list:
                    # Check each type against the avoid list, ensuring case-insensitive comparison
                    if exercise_type.lower() in rules.get('avoid', []):
                        logging.info(f"Exercise {exercise_type} avoided for health condition {condition}")
                        return False  # Ret

        for muscle in involved_muscles:
            if muscle.lower() in condition_lower.replace(' problems', ''):
                logging.info(f"Exercise involving {muscle} muscle avoided due to health condition {condition}")
                return False

    return suitable

def health_based_recommendations_for_single_user(user_id, user_profiles_df, all_exercises):
    user_profile = user_profiles_df.loc[user_profiles_df['UserID'] == user_id].iloc[0] if not user_profiles_df.loc[user_profiles_df['UserID'] == user_id].empty else None
    if user_profile is None:
        logging.error(f"No profile found for user_id {user_id}")
        return []

    user_recommendations = []
    health_conditions = user_profile.get('HealthConditions', [])

    for index, exercise in all_exercises.iterrows():
        exercise_types = [exercise['Type de Exercice']] if 'Type de Exercice' in exercise else []
        involved_muscles = exercise['muscles_involved'] if 'muscles_involved' in exercise else []

        if is_suitable_for_health_condition(health_conditions, exercise_types, involved_muscles):
            user_recommendations.append(exercise['_id'])

    return user_recommendations


In [14]:
##Second version
import logging

def combine_recommendations(user_id, user_profiles, train_set, exercise_encoded_data, user_encoded_data, all_exercise_details):
    weights = {'content': 0.5, 'health': 0.3, 'collaborative': 0.2}
    combined_scores = pd.DataFrame()
    id_to_exercise = all_exercise_details.set_index('_id').to_dict(orient='index')

    # Make sure user_profiles is a DataFrame
    user_profiles_df = pd.DataFrame(user_profiles)

    # Get the profile for the specific user_id
    user_profile = user_profiles_df[user_profiles_df['UserID'] == user_id]
    #days_per_week = user_profile['Days/week'].values[0] if not user_profile.empty else 4

    # Use the new health-based recommendations for single user
    health_filtered_ids = health_based_recommendations_for_single_user(user_id, user_profile, all_exercise_details)

    # Calculate health scores
    if health_filtered_ids:
        health_scores = pd.DataFrame({
            'exercise_id': health_filtered_ids,
            'score': [weights['health'] / len(health_filtered_ids)] * len(health_filtered_ids)
        })
        combined_scores = pd.concat([combined_scores, health_scores])
        
        # Generate content-based recommendations from health filtered IDs
        exercise_encoded_subset = exercise_encoded_data[exercise_encoded_data['_id'].isin(health_filtered_ids)]
        content_based_ids = recommend_based_on_content(user_id, user_encoded_data, exercise_encoded_subset)
        content_scores = pd.DataFrame({
            'exercise_id': content_based_ids,
            'score': [weights['content'] / len(content_based_ids) for _ in range(len(content_based_ids))]
        })
        combined_scores = pd.concat([combined_scores, content_scores])

    # Apply collaborative filtering
    if user_id in train_set['user_id'].values:
        collaborative_ids = generate_item_content_recommendations(user_id, train_set, exercise_encoded_data)
        collaborative_scores = pd.DataFrame({
            'exercise_id': collaborative_ids,
            'score': [weights['collaborative'] / len(collaborative_ids)] * len(collaborative_ids)
        })
        combined_scores = pd.concat([combined_scores, collaborative_scores])
    
    if combined_scores.empty:
        return []

    # Sum scores by exercise_id and sort
    final_recommendations = combined_scores.groupby('exercise_id')['score'].sum().reset_index()
    final_recommendations = final_recommendations.sort_values(by='score', ascending=False)

    # Return top recommendations
    top_recommendations = final_recommendations['exercise_id'].tolist()
    detailed_exercises = []
    for exercise_id in top_recommendations:
        if exercise_id in all_exercise_details.set_index('_id').index:
            exercise_details = all_exercise_details.set_index('_id').loc[exercise_id].to_dict()
            exercise_details['_id'] = exercise_id  # Ensure each exercise dict has its '_id' included
            detailed_exercises.append(exercise_details)

    return detailed_exercises
    
# Ensure logging is configured
logging.basicConfig(level=logging.INFO)




## goal_to_exercise_type_mapping is correct???

In [15]:
def exercise_fit_user_goals(exercise, user_data):
    """
    Check if the exercise aligns with the user's fitness goals.
    """
    goal_to_exercise_type_mapping = {
        "weight loss": ['cardio', 'hiit', 'burn fat', 'losing weight'],
        "muscle gain": ['bodybuilder', 'weight training', 'strength', 'powerlifting'],
        "endurance improvement": ['cardio', 'crossfit', 'competition sportsman'],
        "flexibility": ['yoga', 'stretching', 'calisthenics'],
        "strength": ['strength', 'powerlifting', 'strongman', 'olympic weightlifting']
    }

    # Normalize user goals to lowercase
    user_goals = [goal.lower() for goal in user_data['Goal']]

    # Check if any of the exercise types map to the user's goals
    for user_goal in user_goals:
        for exercise_type in exercise.get('Type de Exercice', []):
            # Normalize exercise type to lowercase
            normalized_exercise_type = exercise_type.lower().replace('type_', '')
            if normalized_exercise_type in goal_to_exercise_type_mapping.get(user_goal, []):
                return True

    return False
def estimate_duration(exercise, user_fitness_level=None):
   
    # Base durations in minutes for one round or set of the exercise
    base_duration = {
        'cardio': 5,  # e.g., jogging or cycling
        'strength': 5,  # e.g., weightlifting set
        'stretching': 5,  # e.g., stretch hold
        'HIIT': 2,  # e.g., one HIIT interval
        'yoga': 5  # e.g., one yoga pose or flow
    }

    # Intensity factors adjust the duration inversely; higher intensity = shorter duration
    intensity_factor = {
        'beginner': 1.2,
        'intermediate': 1.0,
        'advanced': 0.8
    }

    # Fitness level adjustments
    fitness_level_adjustment = {
        'beginner': 0.8,
        'intermediate': 1.0,
        'advanced': 1.2
    }
    default_duration = 5

    # Calculate the duration
    exercise_type = exercise.get('Type de Exercice', 'none')
    exercise_intensity = exercise.get('Level', 'intermediate')
    
    # Check if the type or intensity is a list, and if so, take the first item
    if isinstance(exercise_type, list):
        exercise_type = exercise_type[0] if exercise_type else 'none'
    if isinstance(exercise_intensity, list):
        exercise_intensity = exercise_intensity[0] if exercise_intensity else 'intermediate'

    # Calculation with safeguarded types
    duration = base_duration.get(exercise_type, default_duration) * \
               intensity_factor.get(exercise_intensity, 1.0) * \
               fitness_level_adjustment.get(user_fitness_level, 1.0)
    return int(duration)

In [16]:
def can_fit_in_time(exercise, remaining_time):
    """
    Determine if the exercise can fit within the remaining time allocated for a workout phase.

    Args:
        exercise (dict): The exercise to evaluate.
        remaining_time (int): The remaining time in minutes for the workout phase.

    Returns:
        bool: True if the exercise can fit within the remaining time, False otherwise.
    """
    # Assume 'estimate_duration' returns the duration of the exercise in minutes
    duration = estimate_duration(exercise)
    return duration <= remaining_time


In [17]:
def ensure_unique_exercises(warm_up, main, cool_down, exercise_lookup):
    fallback_warm_up_ids=[45,118,176,185,197,204,224,367,411,413]
    fallback_cool_down_ids=[635,651,768,798,808,938,976,1001,1025,1071,1079]
    main_ids = {ex['_id'] for ex in main}  # Use a set for quick lookup of main workout exercise IDs.

    # Filter out exercises from warm-up and cool-down if they appear in the main workout
    filtered_warm_up = [ex for ex in warm_up if ex['_id'] not in main_ids]
    filtered_cool_down = [ex for ex in cool_down if ex['_id'] not in main_ids]

    # Use fallback IDs to fetch exercises if the filtered lists are empty
    def find_exercise_by_id(exercise_id, exercise_list):
        return next((exercise for exercise in exercise_list if exercise['_id'] == exercise_id), None)

    # Use fallback IDs to fetch exercises if the filtered lists are empty
    if not filtered_warm_up:
        while not filtered_warm_up:
            random_id = random.choice(fallback_warm_up_ids)
            filtered_warm_up = [ex for ex in exercise_lookup if ex['_id'] == random_id]

    # If cool-down list is empty, randomly select exercises until a suitable one is found
    if not filtered_cool_down:
        while not filtered_cool_down:
            random_id = random.choice(fallback_cool_down_ids)
            filtered_cool_down = [ex for ex in exercise_lookup if ex['_id'] == random_id]

    return filtered_warm_up, main, filtered_cool_down


## Warmup & cool down & additional exercices ???

In [18]:
def create_daily_workout(user_id, recommended_exercises, user_data, focus_muscle):
    workout_plan = {'Warm-Up': [], 'Main Workout': [], 'Cool-Down': []}
    time_allocated = {'Warm-Up': 10, 'Cool-Down': 10}  # Increased time allocation

    # Define suitable warm-up and cool-down types
    warm_up_types = ['cardio', 'stretching','strength']
    cool_down_types = ['stretching']

    # Filter exercises by muscle group focus and then by suitable type
    filtered_exercises = [ex for ex in recommended_exercises if focus_muscle in ex['muscles_involved']]
    warm_up_exercises = [
        ex for ex in filtered_exercises
        if any(t in warm_up_types for t in ex.get('Type de Exercice', [])) and (
            ex.get('Level', 'beginner') if 'strength' in ex.get('Type de Exercice', []) else
            ex.get('Level', 'intermediate') in ['intermediate', 'advanced'] if 'stretching' in ex.get('Type de Exercice', []) else
            True  # This keeps cardio exercises regardless of level
        )
    ]

    # Filter cool-down exercises to include those where any type in 'Type de Exercice' list is a cool-down type
    # and level is 'beginner'
    cool_down_exercises = [
        ex for ex in filtered_exercises
        if any(t in cool_down_types for t in ex.get('Type de Exercice', [])) and ex.get('Level', 'beginner') == 'beginner'
    ]

    # Filter main exercises to be all other types excluding those designated for cool-down
    # Ensure no overlap with cool-down types
    main_exercises = [
        ex for ex in filtered_exercises
        if not any(t in cool_down_types for t in ex.get('Type de Exercice', [])) and
        exercise_fit_user_goals(ex, user_data)
    ]
    
    logging.debug(f"Filtered warm-up exercises: {warm_up_exercises}")
    logging.debug(f"Filtered cool-down exercises: {cool_down_exercises}")

    # Function to add exercise to phase if time permits
    def add_exercise_to_phase(phase, exercises, time_left):
        for exercise in exercises:
            duration = estimate_duration(exercise, user_data['FitnessLevel'])
            if time_left >= duration:
                workout_plan[phase].append(exercise)
                return time_left - duration
        return time_left

    # Ensure exercises are unique across the phases
    warm_up_exercises, main_exercises, cool_down_exercises = ensure_unique_exercises(warm_up_exercises, main_exercises, cool_down_exercises, recommended_exercises)

    # Populate phases
    time_allocated['Warm-Up'] = add_exercise_to_phase('Warm-Up', warm_up_exercises, time_allocated['Warm-Up'])
    time_allocated['Cool-Down'] = add_exercise_to_phase('Cool-Down', cool_down_exercises, time_allocated['Cool-Down'])

    # Handle main workout similarly, considering user goals and health conditions if applicable
    main_workout_time = user_data['TimeAvailable'] - sum(time_allocated.values())
    for exercise in main_exercises:
        if main_workout_time < 0:
            break  # Stop if main workout time exceeds available time
        if can_fit_in_time(exercise, main_workout_time):  # Assuming this function checks user goals/health
            workout_plan['Main Workout'].append(exercise)
            main_workout_time -= estimate_duration(exercise, user_data['FitnessLevel'])

    # If there's still time remaining, add more exercises like cardio or HIIT
    if main_workout_time > 0:
        additional_exercises = [
            ex for ex in filtered_exercises
            if ex.get('Type de Exercice') in ['cardio', 'HIIT'] 
            
        ]
        while main_workout_time > 0 and additional_exercises:
            selected_exercise = random.choice(additional_exercises)
            if can_fit_in_time(selected_exercise, main_workout_time):
                workout_plan['Main Workout'].append(selected_exercise)
                main_workout_time -= estimate_duration(selected_exercise, user_data['FitnessLevel'])
            else:
                additional_exercises.remove(selected_exercise)

    return workout_plan


In [19]:
#version 2
import logging
from itertools import cycle
import pandas as pd

def create_monthly_workout_plan(user_id, user_profiles, all_exercise_details):
    logging.basicConfig(level=logging.DEBUG)
    
    # Fetch user data
    if isinstance(user_profiles, list):
        user_profiles = pd.DataFrame(user_profiles)

    user_data = user_profiles.loc[user_profiles['UserID'] == user_id]
    if user_data.empty:
        logging.error(f"User with ID {user_id} not found.")
        return {}
    user_data = user_data.iloc[0]

    # Your combined exercise recommendations based on the user profile
    recommended_exercises = combine_recommendations(user_id, user_profiles, user_interaction, process_encode_exercice_data, user_encoded, all_exercise_details)
    if not recommended_exercises:
        logging.warning("No exercises were recommended.")
        return {}

    muscle_focus_set = set(muscle for exercise in recommended_exercises for muscle in exercise['muscles_involved'])
    muscle_focus_cycle = cycle(muscle_focus_set)
    
    # Define a repeating weekly pattern for workout days
    workouts_per_week = user_data['Days/week']
    pattern = [1] * workouts_per_week + [0] * (7 - workouts_per_week)
    day_pattern = cycle(pattern)
    
    # Create an empty monthly workout plan
    days_in_month = 30
    monthly_workout_plan = {}
    for day in range(1, days_in_month + 1):
        if next(day_pattern):  # Check the pattern to determine workout day
            while True:
                focus_muscle = next(muscle_focus_cycle)
                daily_routine = create_daily_workout(user_id, recommended_exercises, user_data, focus_muscle)
                main_workout_size = len(daily_routine.get('Main Workout', []))
                if main_workout_size >= 3:
                    monthly_workout_plan[day] = daily_routine
                    break
                logging.debug(f"Skipping muscle {focus_muscle} with only {main_workout_size} exercises.")
        else:
            monthly_workout_plan[day] = "Rest"
            logging.debug(f"Rest day scheduled for day {day}")

    return monthly_workout_plan
monthly_workout_plan = create_monthly_workout_plan(2, processed_user_data, processed_exercise_data)



In [20]:
plan_json = json.dumps(monthly_workout_plan, indent=4)
print(plan_json)  # Print JSON string

# Optionally, save to a JSON file
with open('monthly_workout_plan.json', 'w') as f:
    json.dump(monthly_workout_plan, f, indent=4)

{
    "1": {
        "Warm-Up": [
            {
                "name": "Cable rope pull-through deadlift",
                "Equipement": [
                    "cable"
                ],
                "muscles_involved": [
                    "glutes"
                ],
                "image_src_note": {
                    "title": "Cable rope pull-through deadlift Images",
                    "src": [
                        "https://www.bodybuilding.com/images/2020/xdb/originals/2019-xdb-112c-cable-rope-pull-through-deadlift-m1.jpg",
                        "https://www.bodybuilding.com/images/2020/xdb/originals/2019-xdb-112c-cable-rope-pull-through-deadlift-m2.jpg"
                    ]
                },
                "Category": NaN,
                "Steps": {
                    "Images": "https://artifacts.bbcomcdn.com/@bbcom/exercises-app/2.1.2/img/guide-14.gif"
                },
                "Tips ": NaN,
                "Level": "intermediate",
                "Type

In [21]:
def format_workout_plan(monthly_workout_plan):
    formatted_plan = ""
    days_per_week = 7  # Standard 7-day week

    # Calculate the number of weeks in the monthly plan assuming 30 days
    total_weeks = (len(monthly_workout_plan) + days_per_week - 1) // days_per_week

    # Iterate through each week and each day within the week
    for week in range(total_weeks):
        formatted_plan += f"Week {week + 1}:\n"
        for day in range(1, days_per_week + 1):
            current_day = week * days_per_week + day
            if current_day > len(monthly_workout_plan):
                break  # Stop if we exceed the number of days in the month

            day_plan = monthly_workout_plan.get(current_day, "Rest")
            formatted_plan += f"  Day {day}:\n"
            if isinstance(day_plan, dict):  # Check if the day's plan is a dictionary
                for phase, exercises in day_plan.items():
                    formatted_plan += f"    {phase}:\n"
                    for exercise in exercises:
                        formatted_plan += f"      Exercise ID: {exercise['_id']}, Name: {exercise['name']}\n"
            else:
                formatted_plan += f"    {day_plan}\n"  # Handle the case where day_plan is a string (e.g., 'Rest')

    return formatted_plan
display_output = format_workout_plan(monthly_workout_plan)
print(display_output)

Week 1:
  Day 1:
    Warm-Up:
      Exercise ID: 2407, Name: Cable rope pull-through deadlift
    Main Workout:
      Exercise ID: 2177, Name: HM Running in Place
      Exercise ID: 232, Name: Stepper
      Exercise ID: 242, Name: Power SLED (power trolley)
      Exercise ID: 226, Name: Squat Squat
      Exercise ID: 214, Name: Front slits with bar
      Exercise ID: 285, Name: Kettlebell Swing
      Exercise ID: 197, Name: rowing machine
    Cool-Down:
      Exercise ID: 1347, Name: Seated Glute Stretch
  Day 2:
    Warm-Up:
      Exercise ID: 2390, Name: UP Hamstring Curl
    Main Workout:
      Exercise ID: 232, Name: Stepper
      Exercise ID: 242, Name: Power SLED (power trolley)
      Exercise ID: 214, Name: Front slits with bar
      Exercise ID: 285, Name: Kettlebell Swing
      Exercise ID: 367, Name: Assault Bike
      Exercise ID: 1035, Name: Sled push-
    Cool-Down:
      Exercise ID: 808, Name: Standing leg swing
  Day 3:
    Warm-Up:
      Exercise ID: 2398, Name: UP Cab